In [263]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import json
import requests
import json

In [264]:
def strip_price(price):
    rtn = ''
    aGroup = '0123456789.'
    for i in price:
        if i in aGroup:
            rtn += i
    return rtn


In [265]:
# returns 2 list of http links separated by availability
def luckyGunner_availability(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body
    products = soup.find_all('div', class_='main')

    
    # list of all handgun ammo seperated by availability
    products = soup.find_all('div', class_='block-content block-no-padding')
    bulletsList = str(products).split('<div class="main-category-cols">')
    inStock = bulletsList[1]
    outofStock = bulletsList[2]

    inStockScrapedLinks = []
    outofStockScrapedLinks = []
    
    inStock = inStock.split("</li>")
    for inStockItems in inStock:
        inStockScrapedLinks.append(re.findall("(?P<url>https?://[^\s]+)", str(inStockItems)))


    outofStock = outofStock.split("</li>")
    for outofStockItems in outofStock:
        outofStockScrapedLinks.append(re.findall("(?P<url>https?://[^\s]+)", str(outofStockItems)))

    inStockCleanLinks = []
    outofStockCleanLinks = []
    

    for i in inStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                    
                else:
                    inStockCleanLinks.append(cleanLink)
                    break
                
    for i in outofStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                else:
                    outofStockCleanLinks.append(cleanLink)
                    break
    return inStockCleanLinks, outofStockCleanLinks

In [266]:
def scrapeHandGunAmmo(link):
    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
        
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body
    detailPageLinks = []
    links = soup.find_all('h3', class_='product-name')
    detailProduct = soup.find_all('div', class_='product-shop')

    dictLinks = []
    for i in links:
        x = i.find('a')
        detailPageLinks = x['href']
        dictLinks.append(detailPageLinks)
    
    names = []
    prices = []
    calibers = []
    try:
        calibers.append(soup.find('div', class_='category-title').text)
    except:
        print(link)

    linkIndex = 0
    for product in detailProduct:
        aDict = {'name': names, 'prices': prices, 'links': dictLinks, 'caliber': calibers}

        # scrape names
        name = product.find('a')
        name = name.find('span').text
        aDict['name'] = name
        

        # scrape prices
        price = product.find('div', class_='price-box')
        if "Special Price" in str(price):
            price = price.find('span', class_='price')
            aDict['prices'] = strip_price(price.text)
        else:
            if 'As low as' in str(price):
                price = (price.find('span', class_='price'))
                aDict['prices'] = strip_price(price.text)
            else:
                try:
                    prices.append(price.text)
                    aDict['prices'] = strip_price(price.text)
                except:
                    prices.append("NOPE")
                    aDict['prices'] = "out of stock"

        aDict['links'] = dictLinks[linkIndex]
        linkIndex += 1
        try:
            aDict['caliber'] = calibers[0].replace('\n', '')
        except:
            print(link)
        print(aDict)
        print("_________________________-\n")
        
    
    


In [267]:
link = "https://www.luckygunner.com/handgun"
x = luckyGunner_availability(link)

for i in x[0]:
    scrapeHandGunAmmo(i)
    print("++++++++++++++++++++++++++++++++++++++++++++++++")
# 9mm - 124 Grain FMJ - Blazer Brass  - 1000 Rounds

{'name': '25 ACP - 50 Grain FMJ - Sellier & Bellot - 50 Rounds', 'prices': '34.00', 'links': 'https://www.luckygunner.com/25-acp-50-gr-fmj-sellier-bellot-50-rounds', 'caliber': '25 ACP Ammo'}
_________________________-

{'name': '25 ACP - 50 Grain FMJ - Magtech - 50 Rounds', 'prices': '37.00', 'links': 'https://www.luckygunner.com/magtech-25-acp-ammo-for-sale-25acp50fmjmt-50', 'caliber': '25 ACP Ammo'}
_________________________-

++++++++++++++++++++++++++++++++++++++++++++++++
{'name': '30 Luger - 93 gr SJSP - Fiocchi - 50 Rounds', 'prices': '41.00', 'links': 'https://www.luckygunner.com/30-luger-93-gr-sjsp-fiocchi-50-rounds', 'caliber': '30 Luger Ammo'}
_________________________-

++++++++++++++++++++++++++++++++++++++++++++++++
{'name': '32 ACP - 71 Grain FMJ - PMC - 50 Rounds', 'prices': '40.00', 'links': 'https://www.luckygunner.com/32-auto-71-gr-fmj-pmc-50-rounds', 'caliber': '32 ACP Ammo'}
_________________________-

{'name': '32 ACP - 73 Grain FMJ - Fiocchi - 50 Rounds', 'price